In [1]:
!pip install streamlit

   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 5.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.1 MB 3.8 MB/s eta 0:00:03
   ---------- ----------------------------- 2.6/10.1 MB 4.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/10.1 MB 4.5 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.1 MB 4.5 MB/s eta 0:00:02
   -------------------- ------------------- 5.2/10.1 MB 4.2 MB/s eta 0:00:02
   ------------------------ --------------- 6.3/10.1 MB 4.4 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.1 MB 4.6 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/10.1 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.1 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 4.7 MB/s  0:00:02
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ----------

In [3]:
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle between 3 points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start webcam
cap = cv2.VideoCapture(0)

counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate elbow angles
            left_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
            right_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
            avg_angle = (left_angle + right_angle) / 2

            # Push-up logic
            if avg_angle < 70:
                stage = "down"
            if avg_angle > 160 and stage == 'down':
                stage = "up"
                counter += 1

            # Display info
            cv2.rectangle(image, (0, 0), (260, 73), (245, 117, 16), -1)
            cv2.putText(image, 'REPS', (15, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, 'STAGE', (100, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, stage if stage else '-', 
                        (100, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        # Draw landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Push-up Counter', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
